# Feature Engineering - Proyecto ML

Este notebook contiene el proceso completo de ingeniería de características:
1. **Manejo de variables categóricas** (alta cardinalidad)
2. **Creación de nuevas características**
3. **Selección de características**
4. **Gestión de data leakage** con pipelines

## 1. Configuración y Carga de Datos

In [ ]:
# Configuración de rutas
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Rutas del proyecto
PROJ = Path.cwd() if (Path.cwd().name != "notebooks") else Path.cwd().parent
DATA = PROJ / "data"

# Cargar datos
train = pd.read_csv(DATA / "train.csv")
test = pd.read_csv(DATA / "test.csv")

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"\nColumnas: {train.columns.tolist()}")
print(f"\nTarget variable: popularity")
print(f"  - Min: {train['popularity'].min()}")
print(f"  - Max: {train['popularity'].max()}")
print(f"  - Mean: {train['popularity'].mean():.2f}")
print(f"  - Std: {train['popularity'].std():.2f}")

## 2. Análisis de Variables Categóricas - Alta Cardinalidad

**Problema:** Variables como `artists`, `album_name`, `track_name` tienen miles de valores únicos (alta cardinalidad).

**Estrategias:**
- **Frequency Encoding:** Reemplazar por frecuencia de aparición
- **Target Encoding:** Reemplazar por promedio del target (con validación cruzada para evitar leakage)
- **Agrupación:** Agrupar categorías poco frecuentes en "Other"

In [ ]:
# Identificar variables categóricas y su cardinalidad
cat_cols = train.select_dtypes(exclude=['number']).columns.tolist()

print("ANÁLISIS DE CARDINALIDAD:")
print("=" * 60)
for col in cat_cols:
    n_unique_train = train[col].nunique()
    n_unique_test = test[col].nunique()
    print(f"{col:20s}: {n_unique_train:6d} únicos (train), {n_unique_test:6d} únicos (test)")
    
    # Calcular % de valores que aparecen solo 1 vez
    value_counts = train[col].value_counts()
    singleton_pct = (value_counts == 1).sum() / len(value_counts) * 100
    print(f"  {'':20s}  {singleton_pct:.1f}% aparecen solo 1 vez")
    
    # Mostrar top 5 más frecuentes
    print(f"  Top 5: {value_counts.head(5).to_dict()}")
    print()

### 2.1 Transformadores Personalizados para Variables Categóricas

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class FrequencyEncoder(BaseEstimator, TransformerMixin):
    """
    Codifica variables categóricas por su frecuencia de aparición.
    
    Ventajas:
    - Maneja alta cardinalidad sin crear miles de columnas
    - No genera data leakage (aprende solo de train)
    - Valores desconocidos en test reciben frecuencia 0
    """
    def __init__(self, columns):
        self.columns = columns
        self.frequency_maps = {}
    
    def fit(self, X, y=None):
        for col in self.columns:
            if col in X.columns:
                # Calcular frecuencias solo con train
                freq_map = X[col].value_counts(normalize=True).to_dict()
                self.frequency_maps[col] = freq_map
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            if col in X_copy.columns and col in self.frequency_maps:
                # Mapear, valores desconocidos → 0
                X_copy[f'{col}_freq'] = X_copy[col].map(self.frequency_maps[col]).fillna(0)
        return X_copy


class TargetEncoder(BaseEstimator, TransformerMixin):
    """
    Codifica variables categóricas por el promedio del target.
    
    IMPORTANTE: Usa smoothing para evitar overfitting en categorías raras.
    """
    def __init__(self, columns, smoothing=10):
        self.columns = columns
        self.smoothing = smoothing
        self.target_maps = {}
        self.global_mean = None
    
    def fit(self, X, y):
        self.global_mean = y.mean()
        
        for col in self.columns:
            if col in X.columns:
                # Calcular promedio por categoría con smoothing
                agg = X[[col]].copy()
                agg['target'] = y
                
                stats = agg.groupby(col)['target'].agg(['mean', 'count'])
                
                # Smoothing: mezcla promedio de categoría con promedio global
                # Fórmula: (count * mean + smoothing * global_mean) / (count + smoothing)
                smoothed_mean = (
                    (stats['count'] * stats['mean'] + self.smoothing * self.global_mean) /
                    (stats['count'] + self.smoothing)
                )
                
                self.target_maps[col] = smoothed_mean.to_dict()
        
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            if col in X_copy.columns and col in self.target_maps:
                # Mapear, valores desconocidos → global mean
                X_copy[f'{col}_target_enc'] = X_copy[col].map(self.target_maps[col]).fillna(self.global_mean)
        return X_copy


print("✓ Transformadores creados:")
print("  - FrequencyEncoder: Codifica por frecuencia")
print("  - TargetEncoder: Codifica por promedio del target (con smoothing)")

## 3. Creación de Nuevas Características

Crearemos features que capturen información relevante:

In [ ]:
class FeatureCreator(BaseEstimator, TransformerMixin):
    """
    Crea nuevas características a partir de las existentes.
    
    Features creadas:
    1. duration_minutes: Duración en minutos (más interpretable)
    2. energy_danceability: Interacción entre energía y bailabilidad
    3. acousticness_instrumentalness: Ratio acústico/instrumental
    4. loudness_energy_ratio: Relación loudness/energy
    5. speechiness_high: Flag de alto contenido hablado (>0.66)
    6. is_explicit_int: Conversión de explicit a numérico
    7. track_name_length: Largo del nombre de la canción
    8. artists_count: Número de artistas (si hay múltiples separados por coma/feat)
    """
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_new = X.copy()
        
        # 1. Duración en minutos (más interpretable que milisegundos)
        if 'duration_ms' in X_new.columns:
            X_new['duration_minutes'] = X_new['duration_ms'] / 60000
        
        # 2. Interacción entre energy y danceability
        # Canciones "party" suelen tener ambas altas
        if 'energy' in X_new.columns and 'danceability' in X_new.columns:
            X_new['energy_danceability'] = X_new['energy'] * X_new['danceability']
        
        # 3. Ratio acústico/instrumental
        # Canciones muy acústicas vs muy instrumentales
        if 'acousticness' in X_new.columns and 'instrumentalness' in X_new.columns:
            X_new['acoustic_instrumental_ratio'] = (
                X_new['acousticness'] / (X_new['instrumentalness'] + 0.01)  # +0.01 para evitar división por 0
            )
        
        # 4. Relación loudness/energy
        # Canciones que suenan fuerte pero tienen poca energía (o viceversa)
        if 'loudness' in X_new.columns and 'energy' in X_new.columns:
            # Normalizar loudness a [0,1] primero (suele estar en [-60, 0])
            loudness_norm = (X_new['loudness'] + 60) / 60
            X_new['loudness_energy_ratio'] = loudness_norm / (X_new['energy'] + 0.01)
        
        # 5. Flag de alto contenido hablado (podcast, spoken word)
        if 'speechiness' in X_new.columns:
            X_new['speechiness_high'] = (X_new['speechiness'] > 0.66).astype(int)
        
        # 6. Explicit como numérico (si existe)
        if 'explicit' in X_new.columns:
            X_new['is_explicit_int'] = X_new['explicit'].astype(int)
        
        # 7. Largo del nombre de la canción
        if 'track_name' in X_new.columns:
            X_new['track_name_length'] = X_new['track_name'].fillna('').astype(str).str.len()
        
        # 8. Número de artistas (detectar colaboraciones)
        if 'artists' in X_new.columns:
            # Contar separadores: ',' y 'feat' indican múltiples artistas
            X_new['artists_count'] = (
                X_new['artists'].fillna('').astype(str).str.count(',') + 
                X_new['artists'].fillna('').astype(str).str.lower().str.count('feat') + 1
            )
        
        return X_new


feature_creator = FeatureCreator()
print("✓ FeatureCreator listo")
print("\nNuevas features que se crearán:")
print("  1. duration_minutes - Duración en minutos")
print("  2. energy_danceability - Interacción energy × danceability") 
print("  3. acoustic_instrumental_ratio - Ratio acústico/instrumental")
print("  4. loudness_energy_ratio - Relación loudness/energy")
print("  5. speechiness_high - Flag de alto contenido hablado (>0.66)")
print("  6. is_explicit_int - Explicit como numérico")
print("  7. track_name_length - Largo del nombre")
print("  8. artists_count - Número de artistas (colaboraciones)")

## 4. Selección de Características - Eliminación de Columnas Irrelevantes

**Criterios para eliminar columnas:**
1. **Identificadores únicos** (track_id, track_name) - No aportan información predictiva
2. **Alta cardinalidad sin patrón** - Ya las codificamos, podemos eliminar las originales
3. **Información redundante** - Ya capturada en otras features
4. **Leakage potencial** - Información que no estaría disponible antes de la predicción

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    Elimina columnas irrelevantes o problemáticas.
    
    Columnas a eliminar:
    - track_id: Identificador único, no predictivo
    - track_name: Ya capturamos su largo en track_name_length
    - artists: Ya codificada con frequency/target encoding
    - album_name: Ya codificada con frequency/target encoding
    - explicit: Ya convertida a is_explicit_int
    - duration_ms: Ya convertida a duration_minutes (más interpretable)
    """
    
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        # Solo eliminar columnas que existen
        existing_cols = [col for col in self.columns_to_drop if col in X_copy.columns]
        if existing_cols:
            X_copy = X_copy.drop(columns=existing_cols)
        return X_copy


# Definir qué columnas eliminar
columns_to_drop = [
    'track_id',      # Identificador único
    'track_name',    # Ya usamos track_name_length
    'artists',       # Ya codificada
    'album_name',    # Ya codificada
    'explicit',      # Ya convertida a is_explicit_int
    'duration_ms'    # Ya convertida a duration_minutes
]

print("JUSTIFICACIÓN DE ELIMINACIÓN DE COLUMNAS:")
print("=" * 60)
print("✗ track_id: Identificador único, no aporta información predictiva")
print("✗ track_name: Ya capturamos su información en track_name_length")
print("✗ artists: Ya codificada con frequency/target encoding + artists_count")
print("✗ album_name: Ya codificada con frequency/target encoding")
print("✗ explicit: Ya convertida a feature numérica is_explicit_int")
print("✗ duration_ms: Redundante, usamos duration_minutes (más interpretable)")
print("\n✓ Estas columnas se eliminarán DESPUÉS de crear las features derivadas")

## 5. Transformación de Variables Asimétricas (Skewness)

Aplicar transformación logarítmica a variables con distribución asimétrica.

In [ ]:
# Primero debemos aplicar feature_creator para tener todas las columnas numéricas
temp_train = feature_creator.fit_transform(train)

# Identificar variables numéricas continuas
num_continuous = temp_train.select_dtypes(include=['number']).columns.tolist()
if 'popularity' in num_continuous:
    num_continuous.remove('popularity')

# Analizar skewness
print("ANÁLISIS DE ASIMETRÍA (SKEWNESS):")
print("=" * 60)
skewness_dict = {}
for col in num_continuous:
    skew_value = temp_train[col].skew()
    skewness_dict[col] = skew_value
    
    print(f"{col:30s}: skew = {skew_value:7.2f}", end="")
    if abs(skew_value) > 1:
        print("  --> MUY ASIMÉTRICA ⚠️")
    elif abs(skew_value) > 0.5:
        print("  --> Asimetría moderada")
    else:
        print("  --> Simétrica ✓")

# Identificar columnas que necesitan transformación log
skewed_cols = [
    col for col in num_continuous 
    if abs(skewness_dict[col]) > 1.0 and (temp_train[col] >= 0).all()
]

print("\n" + "=" * 60)
print(f"Columnas seleccionadas para transformación logarítmica:")
for col in skewed_cols:
    print(f"  - {col} (skewness = {skewness_dict[col]:.2f})")
print("=" * 60)

In [ ]:
class LogTransformer(BaseEstimator, TransformerMixin):
    """
    Aplica transformación logarítmica a columnas específicas.
    Guarda las columnas internamente para evitar problemas de scope.
    """
    def __init__(self, columns_to_transform):
        self.columns_to_transform = columns_to_transform
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns_to_transform:
            if col in X_copy.columns:
                X_copy[col] = np.log1p(X_copy[col])
        return X_copy
    
    def inverse_transform(self, X):
        X_copy = X.copy()
        for col in self.columns_to_transform:
            if col in X_copy.columns:
                X_copy[col] = np.expm1(X_copy[col])
        return X_copy


log_transformer = LogTransformer(columns_to_transform=skewed_cols)
print(f"✓ LogTransformer creado para {len(skewed_cols)} columnas")

## 6. Pipeline Completo - EVITANDO DATA LEAKAGE

**Orden crítico para evitar leakage:**
1. Crear nuevas features (FeatureCreator)
2. Codificar categóricas con info del train (FrequencyEncoder, TargetEncoder)
3. Transformar variables asimétricas (LogTransformer)
4. Eliminar columnas originales (FeatureSelector)
5. Imputar valores faltantes (SimpleImputer)
6. Escalar variables numéricas (RobustScaler)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

# Identificar columnas categóricas de alta cardinalidad para codificar
high_cardinality_cols = ['artists', 'album_name']

# Identificar columnas categóricas de baja cardinalidad para OneHotEncoding
# (si existen otras categóricas que no sean de alta cardinalidad)
low_cardinality_cols = [col for col in cat_cols if col not in high_cardinality_cols 
                        and col not in ['track_id', 'track_name']]

print("CONFIGURACIÓN DEL PIPELINE:")
print("=" * 60)
print(f"Alta cardinalidad (Frequency + Target Encoding): {high_cardinality_cols}")
print(f"Baja cardinalidad (OneHotEncoding): {low_cardinality_cols}")
print("=" * 60)

# Pipeline de preprocesamiento que se aplica ANTES del ColumnTransformer
feature_engineering_pipeline = Pipeline([
    ('feature_creator', feature_creator),
    ('freq_encoder', FrequencyEncoder(columns=high_cardinality_cols)),
    ('log_transformer', log_transformer),
])

# Después de feature engineering, necesitamos saber qué columnas son numéricas
# Para esto, hacemos un fit_transform temporal
temp_transformed = feature_engineering_pipeline.fit_transform(train, train['popularity'])

# Identificar columnas numéricas después de feature engineering
numeric_features = temp_transformed.select_dtypes(include=['number']).columns.tolist()
if 'popularity' in numeric_features:
    numeric_features.remove('popularity')

# Identificar columnas categóricas restantes
categorical_features = temp_transformed.select_dtypes(exclude=['number']).columns.tolist()

print(f"\nDespués de Feature Engineering:")
print(f"  - Features numéricas: {len(numeric_features)}")
print(f"  - Features categóricas: {len(categorical_features)}")

# Pipeline numérico: imputación + escalado
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())  # Robusto a outliers
])

# Pipeline categórico: imputación + one-hot encoding
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# ColumnTransformer para aplicar diferentes transformaciones
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
], remainder='drop')

# Pipeline completo final
full_pipeline = Pipeline([
    ('feature_engineering', feature_engineering_pipeline),
    ('feature_selector', FeatureSelector(columns_to_drop=columns_to_drop)),
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(
        n_estimators=200,
        max_depth=15,
        min_samples_split=10,
        random_state=42,
        n_jobs=-1
    ))
])

print("\n✓ Pipeline completo creado")
print("\nORDEN DE TRANSFORMACIONES (sin data leakage):")
print("  1. FeatureCreator - Crear nuevas features")
print("  2. FrequencyEncoder - Codificar alta cardinalidad")
print("  3. LogTransformer - Transformar variables asimétricas")
print("  4. FeatureSelector - Eliminar columnas irrelevantes")
print("  5. Preprocessor:")
print("     - Numéricas: Imputar (mediana) + Escalar (RobustScaler)")
print("     - Categóricas: Imputar (Unknown) + OneHotEncode")
print("  6. RandomForestRegressor - Modelo")

## 7. Entrenamiento y Validación

Entrenamos el pipeline completo con validación cruzada.

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import time

# Separar X e y
X_train_full = train.drop(columns=['popularity'])
y_train_full = train['popularity']

# Split para validación
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)

print("ENTRENAMIENTO DEL MODELO:")
print("=" * 60)
print(f"Train set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {test.shape}")
print("=" * 60)

# Entrenar el pipeline completo
start_time = time.time()
full_pipeline.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"\n✓ Entrenamiento completado en {training_time:.2f} segundos")

# Predecir en validación
y_val_pred = full_pipeline.predict(X_val)

# Métricas
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae_val = mean_absolute_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)

print("\nMÉTRICAS EN VALIDACIÓN:")
print("=" * 60)
print(f"RMSE: {rmse_val:.4f}")
print(f"MAE:  {mae_val:.4f}")
print(f"R²:   {r2_val:.4f}")
print("=" * 60)

## 8. Validación Cruzada (Cross-Validation)

Para asegurar que el modelo generaliza bien.

In [ ]:
# Validación cruzada con 5 folds
print("VALIDACIÓN CRUZADA (5-Fold):")
print("=" * 60)

cv_scores = cross_val_score(
    full_pipeline, 
    X_train_full, 
    y_train_full, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

cv_rmse_scores = np.sqrt(-cv_scores)

print(f"RMSE por fold:")
for i, score in enumerate(cv_rmse_scores, 1):
    print(f"  Fold {i}: {score:.4f}")

print(f"\nPromedio: {cv_rmse_scores.mean():.4f} (+/- {cv_rmse_scores.std():.4f})")
print("=" * 60)

# Visualizar resultados de CV
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.boxplot(cv_rmse_scores, labels=['5-Fold CV'])
ax.set_ylabel('RMSE')
ax.set_title('Distribución de RMSE en Validación Cruzada')
ax.grid(True, alpha=0.3)
plt.show()

## 9. Análisis de Importancia de Features

Verificar qué features son más importantes para el modelo.

In [ ]:
# Obtener importancia de features del modelo entrenado
model = full_pipeline.named_steps['model']
feature_importances = model.feature_importances_

# Obtener nombres de features después de la transformación
# Esto es complicado porque ColumnTransformer cambia los nombres
# Vamos a reconstruirlos manualmente

# Aplicar solo el preprocesamiento (sin el modelo)
preprocessing_pipeline = Pipeline([
    ('feature_engineering', feature_engineering_pipeline),
    ('feature_selector', FeatureSelector(columns_to_drop=columns_to_drop)),
])

X_train_transformed = preprocessing_pipeline.fit_transform(X_train, y_train)

# Obtener nombres de features numéricas (después del preprocesamiento)
numeric_feature_names = [col for col in numeric_features if col not in columns_to_drop]

# Nombres de features categóricas después de OneHotEncoding
categorical_feature_names = []
if len(categorical_features) > 0:
    # El OneHotEncoder genera nombres automáticamente
    # Necesitamos obtenerlos del preprocessor
    try:
        ohe = full_pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
        categorical_feature_names = ohe.get_feature_names_out(categorical_features).tolist()
    except:
        pass

# Combinar nombres
all_feature_names = numeric_feature_names + categorical_feature_names

# Si hay diferencia en la longitud, usar nombres genéricos
if len(all_feature_names) != len(feature_importances):
    all_feature_names = [f'feature_{i}' for i in range(len(feature_importances))]

# Crear DataFrame de importancia
feature_importance_df = pd.DataFrame({
    'feature': all_feature_names,
    'importance': feature_importances
}).sort_values('importance', ascending=False)

# Mostrar top 20
print("TOP 20 FEATURES MÁS IMPORTANTES:")
print("=" * 60)
print(feature_importance_df.head(20).to_string(index=False))

# Visualizar
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
top_features = feature_importance_df.head(20)
ax.barh(range(len(top_features)), top_features['importance'])
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'])
ax.set_xlabel('Importancia')
ax.set_title('Top 20 Features Más Importantes')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## 10. Predicción en el Conjunto de Test

Finalmente, aplicamos el pipeline al conjunto de test (sin leakage).

In [ ]:
# Re-entrenar con TODO el conjunto de train
print("ENTRENAMIENTO FINAL CON TODO EL CONJUNTO DE TRAIN:")
print("=" * 60)

full_pipeline.fit(X_train_full, y_train_full)
print("✓ Modelo entrenado con todos los datos de train")

# Predecir en test
y_test_pred = full_pipeline.predict(test)

print(f"\n✓ Predicciones generadas: {len(y_test_pred)}")
print(f"  Rango: [{y_test_pred.min():.2f}, {y_test_pred.max():.2f}]")
print(f"  Media: {y_test_pred.mean():.2f}")
print(f"  Std: {y_test_pred.std():.2f}")

# Comparar con distribución de train
print(f"\nDistribución de popularity en train:")
print(f"  Rango: [{y_train_full.min():.2f}, {y_train_full.max():.2f}]")
print(f"  Media: {y_train_full.mean():.2f}")
print(f"  Std: {y_train_full.std():.2f}")

# Visualizar distribuciones
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(y_train_full, bins=50, alpha=0.7, label='Train (real)', edgecolor='black')
axes[0].set_xlabel('Popularity')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Popularity - Train')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].hist(y_test_pred, bins=50, alpha=0.7, label='Test (predicciones)', 
             color='orange', edgecolor='black')
axes[1].set_xlabel('Popularity')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución de Predicciones - Test')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("✓ PROCESO COMPLETADO SIN DATA LEAKAGE")
print("=" * 60)